In [1]:
import pandas as pd
import requests
import os
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict, Counter

In [2]:
DIR = './data'

In [ ]:
def load_interations(file_name, max_user_cnt=20000, max_user_with_min_interation_cnt = 1000, min_interations = 120):
    count = 0
    data = []
    user_interations = dict()
    users_with_min_interation = set()
    with open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            # Check if 'is_read' is true before processing the entry
            if d.get('is_read', False):  # This correctly checks for True
                user_id = d["user_id"]
                if user_id not in user_interations and len(user_interations) < max_user_cnt :
                    user_interations[user_id] = 1
                else :
                    user_interations[user_id] +=1
                    if user_interations[user_id] > min_interations :
                        users_with_min_interation.add(user_id)
                if user_id in user_interations :
                    data.append(d)
                if len(users_with_min_interation) >= max_user_with_min_interation_cnt :
                    break
    filtered_data = [entry for entry in data if entry["user_id"] in users_with_min_interation]

    return filtered_data

In [12]:
interactions = load_interations(os.path.join(DIR, 'goodreads_interactions_dedup.json.gz'))

In [13]:
print(interactions[:1])

[{'user_id': '8842281e1d1347389f2ab93d60773d4d', 'book_id': '34684622', 'review_id': 'a53868823f065a0e20fd4ae98b820674', 'is_read': False, 'rating': 0, 'review_text_incomplete': '', 'date_added': 'Tue Oct 17 09:40:11 -0700 2017', 'date_updated': 'Tue Oct 17 09:40:12 -0700 2017', 'read_at': '', 'started_at': ''}]


In [14]:
unique_user_count = len(set(item["user_id"] for item in interactions))
unique_book_count = len(set(item["book_id"] for item in interactions))
print("user count: ", unique_user_count, " book count: ", unique_book_count)

user count:  1000  book count:  204839


In [15]:
df = pd.DataFrame(interactions)
user_book_counts = df.groupby('user_id').size()

bucketed_counts = pd.cut(user_book_counts, bins=[0, 50, 100, 200, 500, 1000,  float('inf')])

distribution = bucketed_counts.value_counts(normalize=True) * 100  # In percentage
print(distribution)

(200.0, 500.0]     41.0
(100.0, 200.0]     26.5
(500.0, 1000.0]    20.3
(1000.0, inf]      12.2
(50.0, 100.0]       0.0
(0.0, 50.0]         0.0
Name: proportion, dtype: float64


In [ ]:
unique_book_ids = {item["book_id"] for item in interactions}
df = pd.DataFrame(unique_book_ids, columns=["book_id"])
df.to_csv(os.path.join(DIR, "goodreads_interactions_1k_users_book_ids.csv"), index=False)

In [ ]:
df = pd.DataFrame(interactions)
df.to_json(os.path.join(DIR, "goodreads_interactions_1k_users.json.gz"), orient="records", compression="gzip")